In [1]:
import pandas as pd
import geopandas

In [2]:
vita_sites_df = pd.read_csv("./data/vita_providers_created_by_our_irs_vita_site_downloader_2022-03-24T15 48 05.984215Z.csv")

Per CFA email, we want only sites where `archived=False`.

In [3]:
vita_sites_df = vita_sites_df[vita_sites_df.archived == False]

In [4]:
points = geopandas.GeoSeries.from_wkb(vita_sites_df.coordinates)
vita_gdf = geopandas.GeoDataFrame(vita_sites_df, geometry=points)

In [5]:
vita_gdf.sample(40)

,id,appointment_info,archived,coordinates,created_at,dates,details,hours,irs_id,languages,last_scrape_id,name,updated_at,geometry
210,230,Required,False,0101000020E6100000E9B5D95889B55DC02F4FE78A5216...,2020-01-10T19:32:18.231355,01 FEB 2022 - 26 APR 2022,"2726 E. Hillcrest Drive\nThousand Oaks, CA 913...",NaN,21924,"English,Spanish",1088,Many Mansions Hillcrest Villas,2022-03-20T00:00:44.243045,POINT (-118.83651 34.17439)
4624,4396,Not Required,False,0101000020E61000002C62D8614C0854C08141D2A7552C...,2020-02-25T16:47:11.643031,04 FEB 2022 - 18 APR 2022,"1298 NE 37th Street\nOakland Park, FL 33334\nV...",FRI 3:00PM-6:00PM,24835,"English,Spanish",1088,Hispanic Unity-Ethel M. Gordon Library,2022-03-20T00:01:12.451325,POINT (-80.12966 26.17318)
3946,1050,Not Required,False,0101000020E61000004A0856D5CBDF57C09ACC785BE9B5...,2020-01-10T19:32:25.136154,10 JAN 2022 - 18 APR 2022,6500 Rookin St.\nSuite C-101\nTax Center\nHous...,MON 9:00AM-7:00PM\nTUE 9:00AM-7:00PM\nWED 9:00...,12956,"English,Spanish,Virtual Interpreter Services",1088,BakerRipley Gulfton Sharpstown Campus,2022-03-20T00:00:53.237769,POINT (-95.49682 29.71059)
4125,6820,Required,False,0101000020E6100000AFB0E07EC02B51C024986A662D73...,2022-01-09T00:00:47.883433,01 FEB 2022 - 15 APR 2022,Four Directions Development Corp\n20 Godfrey D...,NaN,17204,English,1088,Wabanaki CA$H - Four Directions,2022-03-20T00:01:14.769765,POINT (-68.68362 44.89982)
1625,493,Required,False,0101000020E610000074EACA6779055CC044300E2E1DC9...,2020-01-10T19:32:20.461998,27 JAN 2022 - 16 APR 2022,914 W Hatcher Road\nBy Appointment only\nPhoen...,NaN,9964,"English,Spanish",1088,PHX Sunnyslope Family Services Center,2022-03-20T00:00:46.846956,POINT (-112.08553 33.57120)
4514,2979,Not Required,False,0101000020E6100000BDFF8F13265052C08FA850DD5C64...,2020-01-10T19:32:41.472509,12 FEB 2022 - 09 APR 2022,Brentwood Public Library\n34 2nd Ave\nSecond A...,SAT 1:00PM-4:30PM,440,English,1088,Brentwood Public Library VITA,2022-03-20T00:01:13.136271,POINT (-73.25232 40.78408)
4057,6799,Not Required,False,0101000020E61000007575C7629B0658C0042159C0042E...,2022-01-09T00:00:40.832702,01 JAN 2022 - 15 APR 2022,"1018 South Third Street\nMabank, TX 75147\nVol...",MON 10:00AM-4:00PM\nWED 10:00AM-4:00PM\nFRI 10...,28434,English,1088,Mabank,2022-03-20T00:00:51.951524,POINT (-96.10323 32.35952)
780,4120,Not Required,False,0101000020E6100000FDD98F14913E53C04451A04FE487...,2020-02-25T16:47:05.882952,01 FEB 2022 - 01 APR 2022,"159 Montezuma St\nLyons, NY 14489\nVolunteer P...",MON 9:00AM-2:00PM\nTUE 9:00AM-2:00PM\nWED 9:00...,3960,English,1088,Wayne County Action Program,2022-03-20T00:01:06.773871,POINT (-76.97760 43.06166)
2117,2166,Required,False,0101000020E61000001763601DC7AC53C044F817416394...,2020-01-10T19:32:34.639381,01 FEB 2022 - 05 APR 2022,"23 East Ave.\nLockport, NY 14094\n716-712-2060...",NaN,23631,English,1088,CCCS of Buffalo Inc- Lockport Public Library,2022-03-20T00:01:04.645868,POINT (-78.69965 43.15928)
2752,1623,Required,False,0101000020E61000005E6919A9F70356C0C217265305AF...,2020-01-10T19:32:30.106713,25 JAN 2022 - 15 APR 2022,"1150 Government St\nSuite 209\nMobile, AL 3660...",NaN,11061,English,1088,Impact Bay Haas Building,2022-03-20T00:00:58.838247,POINT (-88.06199 30.68368)


In [6]:
counties_gdf = geopandas.read_file("./data/cb_2019_us_county_5m/cb_2019_us_county_5m.shp")

In [7]:
vita_gdf = vita_gdf.set_crs(counties_gdf.crs)

### Spatial join of VITA sites and counties

Here we use [`geopandas sjoin`](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins) to merge the Census counties shapefile with the VITA sites. This spatial join finds all VITA sites existing within the geometry of the county.

In [8]:
vita_counties_gdf = counties_gdf.sjoin(vita_gdf)

In [9]:
vita_counties_gdf[pd.isnull(vita_counties_gdf.COUNTYFP)]

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,coordinates,created_at,dates,details,hours,irs_id,languages,last_scrape_id,name,updated_at


In [10]:
vita_counties_gdf = vita_counties_gdf[vita_counties_gdf.archived == False].copy()

Every VITA site was matched to a county.

In [11]:
vita_counties_gdf.groupby('index_right').size().sort_values()

index_right
0       1
3276    1
3277    1
3278    1
3280    1
       ..
1475    1
1476    1
1477    1
1479    1
6297    1
Length: 3150, dtype: int64

And no VITA site appears twice. We're all set!

In [12]:
vita_counties_gdf.geometry

2       POLYGON ((-78.80252 36.21349, -78.80235 36.220...
2       POLYGON ((-78.80252 36.21349, -78.80235 36.220...
2       POLYGON ((-78.80252 36.21349, -78.80235 36.220...
2       POLYGON ((-78.80252 36.21349, -78.80235 36.220...
5       POLYGON ((-83.11274 40.99345, -82.85318 40.993...
                              ...                        
3224    POLYGON ((-96.83451 47.00887, -96.83350 47.010...
3224    POLYGON ((-96.83451 47.00887, -96.83350 47.010...
3225    POLYGON ((-84.52564 37.76950, -84.49596 37.785...
3226    POLYGON ((-66.22683 18.11664, -66.21690 18.143...
3232    POLYGON ((-88.93308 37.22775, -88.93174 37.227...
Name: geometry, Length: 3150, dtype: geometry

In [13]:
# Lat/long of VITA sites for Tableau
vita_counties_gdf['lat_long'] = geopandas.GeoSeries.from_wkb(vita_counties_gdf.coordinates)
vita_counties_gdf['longitude'] = vita_counties_gdf.lat_long.apply(lambda point: point.x)
vita_counties_gdf['latitude'] = vita_counties_gdf.lat_long.apply(lambda point: point.y)
vita_counties_gdf.drop(columns=['lat_long'], inplace=True)

In [14]:
vita_counties_gdf.to_file("vita_sites_by_county.shp")

/var/folders/_j/xzgw3fcs6nn7048q7z34dnmh0000gn/T/ipykernel_22664/669558613.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vita_counties_gdf.to_file("vita_sites_by_county.shp")


In [18]:
pd.DataFrame(vita_counties_gdf[['STATEFP', 'COUNTYFP', 'AFFGEOID', 'GEOID', 'id', 'appointment_info', 'created_at', 'dates', 'details', 'hours', 'irs_id', 'languages', 'name', 'updated_at', 'longitude', 'latitude']]).to_csv("./data/vita_sites_by_county.csv", index=False)

In [16]:
vita_counties_gdf.columns

Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry', 'index_right', 'id', 'appointment_info',
       'archived', 'coordinates', 'created_at', 'dates', 'details', 'hours',
       'irs_id', 'languages', 'last_scrape_id', 'name', 'updated_at',
       'longitude', 'latitude'],
      dtype='object')